In [1]:
import pandas as pd
from decimal import Decimal
from typing import List, Dict, Any
from fastlane_bot.tools.optimizer import CPCArbOptimizer
from dataclasses import asdict
from fastlane_bot.helpers import TradeInstruction

In [2]:
## mock only
best_trade_instructions_dic = [
{'cid': '218',
  'tknin': '0x0-1AD5',
  'amtin': 5309.369973833374,
  'tknout': 'WETH-6Cc2',
  'amtout': -0.09179346660459942},
{'cid': '9868188640707215440437863615521278132277-1',
  'tknin': 'WETH-6Cc2',
  'amtin': 0.0785107723399976,
  'tknout': '0x0-1AD5',
  'amtout': -5363.0},
{'cid': '9868188640707215440437863615521278132278-1',
  'tknin': 'WETH-6Cc2',
  'amtin': 0.0111111111111111,
  'tknout': '0x0-1AD5',
  'amtout': -3000.0},
{'cid': '219',
  'tknin': '0x0-1AD5',
  'amtin': 5309.369973833374,
  'tknout': 'WETH-6Cc2',
  'amtout': -0.09179346660459942},
{'cid': '9868188640707215440437863615521278132280-0',
  'tknin': '0x0-1AD5',
  'amtin': 1000.0,
  'tknout': 'WETH-6Cc2',
  'amtout': -0.001},
{'cid': '9868188640707215440437863615521278132281-0',
  'tknin': '0x0-1AD5',
  'amtin':  2000,
  'tknout': 'WETH-6Cc2',
  'amtout': -0.05},
  {'cid': '220',
  'tknin': '0x0-1AD5',
  'amtin': 5309.369973833374,
  'tknout': 'WETH-6Cc2',
  'amtout': -0.09179346660459942},
{'cid': '9868188640707215440437863615521278132290-1',
  'tknin': 'WETH-6Cc2',
  'amtin': 0.016351323399976,
  'tknout': '0x0-1AD5',
  'amtout': -1234.0},
{'cid': '9868188640707215440437863615521278132291-1',
  'tknin': 'WETH-6Cc2',
  'amtin': 0.022222222222,
  'tknout': '0x0-1AD5',
  'amtout': -2000.0},
]
best_src_token = 'WETH-6Cc2'

In [3]:
def simple_ordering_by_src_token(best_trade_instructions_dic, best_src_token):
    '''
    Reorders a trade_instructions_dct so that all items where the best_src_token is the tknin are before others
    '''
    ordered_trade_instructions_dct = [x for x in best_trade_instructions_dic if x['tknin'] == best_src_token] + [x for x in best_trade_instructions_dic if x['tknin'] != best_src_token]

    return ordered_trade_instructions_dct

In [4]:
### TEST ###
ordered_trade_instructions_dct = simple_ordering_by_src_token(best_trade_instructions_dic, best_src_token)
srctoken_indexes = [i for i,c in list(enumerate(ordered_trade_instructions_dct)) if c['tknin'] == best_src_token]
non_srctoken_indexes = [i for i,c in list(enumerate(ordered_trade_instructions_dct)) if c['tknin'] != best_src_token]
assert max(srctoken_indexes) < min(non_srctoken_indexes)
### TEST ###

In [5]:
def basic_scaling_alternative_to_exact(best_trade_instructions_dic, best_src_token):
    '''
    For items in the trade_instruction_dic scale the amtin by 0.999 if its the src_token, else by 0.99

    NOTE: Since we update the info in the DICTIONARY, this actually voids the info in the DATAFRAME
    '''
    scaled_best_trade_instructions_dic = []
    for x in best_trade_instructions_dic:
        scaled_best_trade_instructions_dic += [{k:v for k,v in x.items()}]

    for i in range(len(scaled_best_trade_instructions_dic)):
        if scaled_best_trade_instructions_dic[i]["tknin"] == best_src_token:
            scaled_best_trade_instructions_dic[i]["amtin"] *= 0.999
        else:
            scaled_best_trade_instructions_dic[i]["amtin"] *= 0.99

    return scaled_best_trade_instructions_dic

In [6]:
### TEST ###
scaled_best_trade_instructions_dic = basic_scaling_alternative_to_exact(best_trade_instructions_dic, best_src_token)
assert sum([x['amtin'] for x in scaled_best_trade_instructions_dic]) < sum([x['amtin'] for x in best_trade_instructions_dic])
### TEST ###

In [7]:
trade_instructions_objects = [TradeInstruction(**ti) for ti in best_trade_instructions_dic]
trade_instructions_objects

[TradeInstruction(cid='218', tknin='0x0-1AD5', amtin=5309.369973833374, tknout='WETH-6Cc2', amtout=-0.09179346660459942, pair_sorting=None, raw_txs=None, custom_data=''),
 TradeInstruction(cid='9868188640707215440437863615521278132277-1', tknin='WETH-6Cc2', amtin=0.0785107723399976, tknout='0x0-1AD5', amtout=-5363.0, pair_sorting=None, raw_txs=None, custom_data=''),
 TradeInstruction(cid='9868188640707215440437863615521278132278-1', tknin='WETH-6Cc2', amtin=0.0111111111111111, tknout='0x0-1AD5', amtout=-3000.0, pair_sorting=None, raw_txs=None, custom_data=''),
 TradeInstruction(cid='219', tknin='0x0-1AD5', amtin=5309.369973833374, tknout='WETH-6Cc2', amtout=-0.09179346660459942, pair_sorting=None, raw_txs=None, custom_data=''),
 TradeInstruction(cid='9868188640707215440437863615521278132280-0', tknin='0x0-1AD5', amtin=1000.0, tknout='WETH-6Cc2', amtout=-0.001, pair_sorting=None, raw_txs=None, custom_data=''),
 TradeInstruction(cid='9868188640707215440437863615521278132281-0', tknin='0x

In [8]:
def _get_trade_dicts_from_objects(trade_instructions: List[TradeInstruction]) -> List[Dict[str, Any]]:
    return [
        {
            "cid": instr.cid + "-" + str(instr.cid_tkn)
            if instr.cid_tkn
            else instr.cid,
            "tknin": instr.tknin,
            "amtin": instr.amtin,
            "tknout": instr.tknout,
            "amtout": instr.amtout,
        }
        for instr in trade_instructions
    ]

In [9]:
def slice_dataframe(df):
    slices = []
    current_pair_sorting = df.pair_sorting.values[0]
    current_slice = []

    for index, row in df.iterrows():
        if row['pair_sorting'] == current_pair_sorting:
            current_slice.append(index)
        else:
            slices.append(df.loc[current_slice])
            current_pair_sorting = row['pair_sorting']
            current_slice = [index]

    slices.append(df.loc[current_slice])

    min_index = []
    for df in slices:
        min_index += [min(df.index.values)]

    
    return list(zip(min_index, slices))


In [10]:
def aggregate_carbon_trades(trade_instructions_objects: List[TradeInstruction]) -> List[TradeInstruction]:
    """
    Aggregate carbon independent IDs and create trade instructions.

    This function takes a list of dictionaries containing trade instructions,
    aggregates the instructions with carbon independent IDs, and creates
    a list of TradeInstruction objects.

    Parameters
    ----------
    trade_instructions : List[TradeInstruction]
        A list of trade instructions as TradeInstruction objects.

    Returns
    -------
    List[TradeInstruction]
        A list of aggregated trade instructions as TradeInstruction objects.

    """
    # Get the indices of the carbon trades
    listti = _get_trade_dicts_from_objects(trade_instructions_objects)
    df = pd.DataFrame(listti)
    df["pair_sorting"] = df.tknin + df.tknout
    df['carbon'] = [True if '-' in df.cid[i] else False for i in df.index]

    carbons = df[df['carbon']].copy()
    nocarbons = df[~df['carbon']].copy()
    nocarbons["raw_txs"] = str([])

    carbons.drop(['carbon'], axis=1, inplace=True)
    nocarbons.drop(['carbon'], axis=1, inplace=True)

    new_trade_instructions_nocarbons = {i: nocarbons.loc[i].to_dict() for i in nocarbons.index}

    result = slice_dataframe(carbons)
    new_trade_instructions_carbons = {min_index:
        {
            "pair_sorting": newdf.pair_sorting.values[0],
            "cid": newdf.cid.values[0],
            "tknin": newdf.tknin.values[0],
            "amtin": newdf.amtin.sum(),
            "tknout": newdf.tknout.values[0],
            "amtout": newdf.amtout.sum(),
            "raw_txs": str(newdf.to_dict(orient="records")),
        }
        for min_index, newdf in slice_dataframe(carbons)}

    new_trade_instructions_carbons.update(new_trade_instructions_nocarbons)
    agg_trade_instructions = []
    for i in sorted(list(new_trade_instructions_carbons.keys())):
        agg_trade_instructions += [TradeInstruction(**new_trade_instructions_carbons[i])]
    return agg_trade_instructions

In [11]:
# this example is different to the example usage in that the dict hasn't been sorted first, 
# I did this so I could verify that the aggregation works on adjacent same-pair only
agg_trade_instructions = aggregate_carbon_trades(trade_instructions_objects)
agg_trade_instructions

[TradeInstruction(cid='218', tknin='0x0-1AD5', amtin=5309.369973833374, tknout='WETH-6Cc2', amtout=-0.09179346660459942, pair_sorting='0x0-1AD5WETH-6Cc2', raw_txs='[]', custom_data=''),
 TradeInstruction(cid='9868188640707215440437863615521278132277-1', tknin='WETH-6Cc2', amtin=0.0896218834511087, tknout='0x0-1AD5', amtout=-8363.0, pair_sorting='WETH-6Cc20x0-1AD5', raw_txs="[{'cid': '9868188640707215440437863615521278132277-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.0785107723399976, 'tknout': '0x0-1AD5', 'amtout': -5363.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}, {'cid': '9868188640707215440437863615521278132278-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.0111111111111111, 'tknout': '0x0-1AD5', 'amtout': -3000.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}]", custom_data=''),
 TradeInstruction(cid='219', tknin='0x0-1AD5', amtin=5309.369973833374, tknout='WETH-6Cc2', amtout=-0.09179346660459942, pair_sorting='0x0-1AD5WETH-6Cc2', raw_txs='[]', custom_data=''),
 TradeInstruction(cid='98681886407072154404378636

# Example Usage

In [12]:
ordered_trade_instructions_dct = simple_ordering_by_src_token(best_trade_instructions_dic, best_src_token)
scaled_best_trade_instructions_dic = basic_scaling_alternative_to_exact(ordered_trade_instructions_dct, best_src_token)

# YOU DONT NEED THIS just a stand-in for _convert_trade_instructions
trade_instructions_objects = [TradeInstruction(**ti) for ti in scaled_best_trade_instructions_dic]

agg_trade_instructions = aggregate_carbon_trades(trade_instructions_objects)
agg_trade_instructions

[TradeInstruction(cid='9868188640707215440437863615521278132277-1', tknin='WETH-6Cc2', amtin=0.12806723364401162, tknout='0x0-1AD5', amtout=-11597.0, pair_sorting='WETH-6Cc20x0-1AD5', raw_txs="[{'cid': '9868188640707215440437863615521278132277-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.0784322615676576, 'tknout': '0x0-1AD5', 'amtout': -5363.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}, {'cid': '9868188640707215440437863615521278132278-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.011099999999999988, 'tknout': '0x0-1AD5', 'amtout': -3000.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}, {'cid': '9868188640707215440437863615521278132290-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.016334972076576026, 'tknout': '0x0-1AD5', 'amtout': -1234.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}, {'cid': '9868188640707215440437863615521278132291-1', 'tknin': 'WETH-6Cc2', 'amtin': 0.022199999999778, 'tknout': '0x0-1AD5', 'amtout': -2000.0, 'pair_sorting': 'WETH-6Cc20x0-1AD5'}]", custom_data=''),
 TradeInstruction(cid='218', tknin='0x0-1AD5', a